In [2]:
import pickle
import numpy as np
import pandas as pd
import cupy as cp
import time
from tqdm import tqdm

## synthetic data(pkl파일)와 data(csv파일) 두 데이터셋에서 각 행을 비교해서 동일한 행이 몇 개인지 확인

## Dataframe

In [3]:

dtypes = {'age':'int','workclass':'category','education':'category','marital_status':'category','occupation':'category','relationship':'category','race':'category',
          'gender':'category','capital_gain':'int','capital_loss':'int','hours_per_week':'int','native_country':'category','income':'category'}
columns = list(dtypes.keys())
data = pd.read_csv('/home/casey/workspace/BigDataSystem/adultSalary.csv', names=columns).astype(dtypes)

data

,age,workclass,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,HS-grad,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [4]:
n = 20000
n = len(data)
data = data[:n]

### df_synthpop

In [5]:
# pkl 파일 경로를 지정합니다.
df_synthpop_pkl = '/home/casey/workspace/BigDataSystem/df_synthpop.pickle'
# 파일을 읽기 모드(rb)로 열어서 데이터를 로드합니다.
with open(df_synthpop_pkl, 'rb') as f:
    synth_data = pickle.load(f)
    
# 데이터를 확인합니다.
synth_data


,age,workclass,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
0,67,?,HS-grad,Married-civ-spouse,?,Wife,White,Female,0,0,4,United-States,<=50K
1,21,Private,Some-college,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
2,55,Self-emp-not-inc,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,0,0,30,United-States,<=50K
3,33,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,65,United-States,>50K
4,53,Local-gov,11th,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,26,Private,10th,Never-married,Craft-repair,Other-relative,Black,Male,0,0,20,United-States,<=50K
48838,33,Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
48839,71,Self-emp-inc,Some-college,Widowed,Exec-managerial,Not-in-family,White,Female,0,0,24,United-States,<=50K
48840,25,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K


In [6]:
import pandas as pd
import time
from tqdm import tqdm

# 데이터를 원본 그대로 유지하면서 인덱스를 재설정하지 않도록 하여 비교
data_list = []
synth_data_list = []

# 원본 데이터와 합성 데이터의 인덱스를 맞추는 과정
data = data.reset_index(drop=True)
synth_data = synth_data.reset_index(drop=True)

# 얘 땜에 index 변경됨
# synth_data = synth_data[:n-int(n*0.1)]
# synth_data = pd.concat([data[:int(n*0.1)], synth_data])

# 행 데이터를 리스트로 변환
stime = time.time()
for i in range(len(synth_data)):
    data_list.append(data.iloc[i, :].values.flatten().tolist())
    synth_data_list.append(synth_data.iloc[i, :].values.flatten().tolist())

cnt = 0
for row1 in tqdm(synth_data_list):
    for row2 in data_list:
        if row1 == row2:
            cnt += 1
            break

print("List 방식 시간:", time.time() - stime)
print("중복률:", cnt / len(synth_data))

# 인덱스 확인
print(data.index.equals(synth_data.index))  # True로 나와야 함


100%|██████████| 48842/48842 [02:25<00:00, 335.71it/s]

List 방식 시간: 147.80453538894653
중복률: 0.4122886040702674
True


### list로 바꿔서 해보기
- 이렇게 하면 원하는 값이 있는지 확인할 수 있음
- DataFrame과 List 속도 차이 : List, array로 바꿔서 사용하면 더 빠름

In [7]:
print(data.index.equals(synth_data.index))

True


### 해시 (set) + 튜플 방식

In [42]:
stime = time.time()

# 1) 원본 행을 튜플로 바꿔 set에 저장
orig_tuples = set(map(tuple, data.values))
# print(data.values)
# print("========")
# print(synth_data.values)
# 2) 합성 행을 튜플로 바꿔서 in 연산
cnt = 0
for row in synth_data.values:
    if tuple(row) in orig_tuples:
        cnt += 1
etime = time.time()
print(f"실행시간: {etime- stime:.2f}s")
print("중복률:", cnt / len(synth_data)) 
# print(cnt)


실행시간: 0.12s
중복률: 0.4122886040702674


### Numba

In [37]:
import numba

# Numba JIT를 사용한 중복 체크 함수 (병렬 처리)
@numba.jit(nopython=True, parallel=True)
def check_duplicates(data, synth_data):
    n_s = len(synth_data)
    n_o = len(data)
    cnt = 0
    for i in numba.prange(n_s):  # 병렬화
        for j in range(n_o):
            if np.all(data[j] == synth_data[i]):
                cnt += 1
                break  # 중복을 찾으면 더 이상 비교하지 않음
    return cnt

# 모든 범주형 데이터를 원핫 인코딩
data_encoded = pd.get_dummies(data)
synth_encoded = pd.get_dummies(synth_data)

# 컬럼이 완전히 같도록 정렬 (중요!)
synth_encoded = synth_encoded.reindex(columns=data_encoded.columns, fill_value=0)
print(synth_encoded.index.equals(data_encoded.index))

# pandas DataFrame을 NumPy 배열로 변환할 때, dtype을 명시적으로 지정 (float32로 변환)
data_numpy = data_encoded.to_numpy(dtype=np.float32)  # float32로 변환
synth_data_numpy = synth_encoded.to_numpy(dtype=np.float32)  # float32로 변환


# 첫 번째 실행 시간 측정 (JIT 컴파일 포함)
stime = time.time()
duplicates_1 = check_duplicates(data_numpy, synth_data_numpy)
etime = time.time()
print(f"첫 번째 실행 시간: {etime - stime:.4f} 초")

# 두 번째 실행 시간 측정 (컴파일된 코드가 캐시됨)
stime = time.time()
duplicates_2 = check_duplicates(data_numpy, synth_data_numpy)
etime = time.time()
print(f"두 번째 실행 시간: {etime - stime:.4f} 초")

# 중복률 출력
print(f"중복률: {duplicates_1 / len(synth_data_numpy):.4f}")


True
첫 번째 실행 시간: 22.5994 초
두 번째 실행 시간: 22.0905 초
중복률: 0.4123


아니 두번째 실행에서 엄청 빨라질 줄 알았더니 왜 안됨?

### Numpy

In [ ]:
stime = time.time()
a = data.to_numpy()
b = synth_data.to_numpy()

cnt = sum([np.any(np.all(row == a, axis=1)) for row in b])

print("Numpy 방식 시간:", time.time() - stime)
print("중복률:", cnt / len(b))
# Numpy 방식 시간: 231.23529720306396
# 중복률: 0.47049670365668894

Numpy 방식 시간: 238.78452682495117
중복률: 0.4122886040702674


### Cupy

In [ ]:
# import pandas as pd
# import cupy as cp
# import time

# # 모든 범주형 데이터를 원핫 인코딩
# data_encoded = pd.get_dummies(data)
# synth_encoded = pd.get_dummies(synth_data)

# # 컬럼이 완전히 같도록 정렬 (중요!)
# synth_encoded = synth_encoded.reindex(columns=data_encoded.columns, fill_value=0)

# # CuPy로 변환 (numpy 배열을 CUDA GPU 메모리로 복사)
# stime = time.time()
# a_gpu = cp.asarray(data_encoded.to_numpy(dtype='float32'))  # cupy 배열 객체 
# b_gpu = cp.asarray(synth_encoded.to_numpy(dtype='float32')) # cupy 배열 객체
# # 중복 체크
# # cnt = 0
# # for i in range(b_gpu.shape[0]):
# #     ## 각 원본 행이 모든 컬럼에서 일치 하는지 -> 원본 중 하나라도 완전 일치 하는 행 있는지
# #     if cp.any(cp.all(b_gpu[i] == a_gpu, axis=1)):  
# #         cnt += 1


# # cp.isin을 사용하여 일치 여부 계산
# matching_rows = cp.isin(a_gpu, b_gpu)
# list = cp.where(cp.isin(a_gpu, b_gpu) == False)[0]
# for x in list:
#     print(x)

# # False인 값만 추출 (a_gpu에서 b_gpu에 포함되지 않는 값)
# non_matching_rows = a_gpu[~matching_rows]

# # 중복률 계산 (중복된 값의 비율)
# unique_non_matching_rows = cp.unique(non_matching_rows)  # 중복 제거된 값들
# non_matching_count = non_matching_rows.shape[0]  # False인 값들의 총 개수
# print(non_matching_count)
# unique_non_matching_count = unique_non_matching_rows.shape[0]  # 중복을 제거한 값들의 개수
# print(unique_non_matching_count)
# print(((1-0.5384615384615384) - 0.47049670365668894)*48842)
# # 중복률 계산
# duplicate_ratio = 1 - (unique_non_matching_count / non_matching_count)
# etime    = time.time()

# print("CuPy 방식 시간:", etime - stime)
# print("중복률:",duplicate_ratio)
# # CuPy 방식 시간: 15.944603443145752
# # 중복률: 0.47049670365668894

1246
1368
1482
1528
1616
1682
1765
1771
1826
2103
2319
2358
2361
2707
3105
3175
3368
3593
3836
4389
4421
4625
4656
4898
5098
5184
5473
5588
6002
6035
6225
6524
6751
6959
7090
7347
7517
7553
7572
7629
7745
8442
8476
8710
8740
8742
9184
9228
9673
9760
10366
10661
10771
10848
10962
10964
11485
11976
12062
12093
12141
12533
12539
12655
12677
12909
13422
13455
13499
13997
14117
14238
14579
14827
15100
15279
15604
15737
15904
16079
16174
16422
16740
17330
17538
17609
17644
17665
17789
18080
18126
18315
18408
18463
18654
18882
19080
19084
19133
19353
19438
19653
19807
19900
20055
20283
20417
20613
20987
21188
21489
21992
22275
22317
22361
22385
22749
22811
23087
23467
23678
23999
24008
24067
24105
24200
24285
24295
24510
24638
24673
24850
24983
25178
25372
25611
25633
25841
26083
26414
26442
26593
26825
27077
27221
27358
27413
27635
27640
28054
28214
28264
28294
28318
28349
28474
29447
29635
29806
30244
30496
30913
31111
31828
31972
32090
32238
32518
32644
32907
32918
32979
33253
33263
33443


In [41]:
import pandas as pd
import cupy as cp
import numpy as np
import time
import hashlib

# 1. 원핫 인코딩
data_encoded = pd.get_dummies(data)
synth_encoded = pd.get_dummies(synth_data)

# 2. 컬럼 통일 및 순서 보장
all_columns = sorted(set(data_encoded.columns).union(set(synth_encoded.columns)))
data_encoded = data_encoded.reindex(columns=all_columns, fill_value=0).astype(np.uint8)
synth_encoded = synth_encoded.reindex(columns=all_columns, fill_value=0).astype(np.uint8)
# 3. 행 단위 해시 함수
def hash_row(row):
    return np.uint64(int(hashlib.md5(row.tobytes()).hexdigest(), 16) & 0xFFFFFFFFFFFFFFFF)

stime = time.time()

# 4. 해시값으로 변환
data_hashes = np.array([hash_row(row) for row in data_encoded.to_numpy()])
synth_hashes = np.array([hash_row(row) for row in synth_encoded.to_numpy()])
mtime = time.time()
# 6. CuPy 비교
a_gpu = cp.asarray(data_hashes)
b_gpu = cp.asarray(synth_hashes)
matched_gpu = cp.isin(b_gpu, a_gpu)
etime = time.time()
# 7. 결과
count = int(cp.sum(matched_gpu))
duplication_ratio = count / len(b_gpu)
print(f"CUPY 실행시간: {etime- stime:.2f}s")
print(f"GPU 메모리에 올린 후, 오로지 계산 실행 시간만!: {etime- mtime:.2f}s")

print("중복된 고유 행 수:", count)
print("중복률:", duplication_ratio)



CUPY 실행시간: 0.14s
GPU 메모리에 올린 후, 오로지 계산 실행 시간만!: 0.02s
중복된 고유 행 수: 20137
중복률: 0.4122886040702674


- numpy : 안에서 C, 포트란?? 으로 돌아서.빠름
- list
- 1) 문제 해결 시간 단축 노력 해보기~
- 2) cython? 은 어려움

In [ ]:
# 1. 범위 계산 함수
def compute_ranges(original_df, dtypes):
    ranges = []
    for col in original_df.columns:
        if dtypes[col] == 'category':
            ranges.append(0)
        else:
            col_data = original_df[col].astype(float)
            ranges.append(col_data.max() - col_data.min())
    return dict(zip(original_df.columns, ranges))

# 2. 원핫 range 계산
def get_onehot_ranges(data_encoded, original_columns, raw_ranges):
    onehot_ranges = []
    for col in data_encoded.columns:
        base_col = col.split('_')[0]
        range_val = raw_ranges.get(base_col, 1)
        onehot_ranges.append(0 if range_val == 0 else 1)
    return np.array(onehot_ranges, dtype=np.float32)

# 3. CuPy 기반 originality 계산
def calculate_originality_score_cupy(origin_np, synth_np, ranges_np):
    origin = cp.asarray(origin_np)
    synth = cp.asarray(synth_np)
    ranges = cp.asarray(ranges_np)
    
    n_s, k = synth.shape
    n_o = origin.shape[0]
    cnt_satisfied = 0
    stime = time.time()

    for i in range(n_s):
        y = synth[i]

        diff = cp.abs(origin - y)
        normed = cp.where(ranges == 0, (origin != y).astype(cp.float32), diff / ranges)
        dists = cp.mean(normed, axis=1)
        d_s = cp.min(dists)
        jj = cp.argmin(dists)

        mask = cp.arange(n_o) != jj
        origin_others = origin[mask]
        ref = origin[jj]

        diff_o = cp.abs(origin_others - ref)
        normed_o = cp.where(ranges == 0, (origin_others != ref).astype(cp.float32), diff_o / ranges)
        dists_o = cp.mean(normed_o, axis=1)
        d_o = cp.min(dists_o)

        if d_s < d_o:
            cnt_satisfied += 1

    etime = time.time()
    score = cnt_satisfied / n_s

    print(f"\n✅ Originality Score (d_s < d_o): {score:.4f}")
    print(f"✔️ {cnt_satisfied} / {n_s} 합성 샘플이 조건 만족")
    print(f"⏱️ 실행 시간: {etime - stime:.2f}초")
    return score


In [ ]:
# 4. ranges 계산 (원본 기준)
raw_ranges = compute_ranges(data, dtypes)

# 5. 원핫 인코딩
data_encoded = pd.get_dummies(data)
synth_encoded = pd.get_dummies(synth_data)

# 6. 컬럼 통일
all_columns = sorted(set(data_encoded.columns).union(set(synth_encoded.columns)))
data_encoded = data_encoded.reindex(columns=all_columns, fill_value=0).astype(np.float32)
synth_encoded = synth_encoded.reindex(columns=all_columns, fill_value=0).astype(np.float32)

# 7. onehot에 대한 range 매핑
ranges_np = get_onehot_ranges(data_encoded, data.columns, raw_ranges)

# 8. NumPy 변환
origin_np = data_encoded.to_numpy()
synth_np = synth_encoded.to_numpy()

# 9. originality score 계산
originality_score = calculate_originality_score_cupy(origin_np, synth_np, ranges_np)

if originality_score >= 0.5:
    print("🎯 Originality 기준 만족")
else:
    print("⚠️ Originality 기준 미달")




✅ Originality Score (d_s < d_o): 0.4479
✔️ 21877 / 48842 합성 샘플이 조건 만족
⏱️ 실행 시간: 174.35초
⚠️ Originality 기준 미달


In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import pickle
import time
from sklearn.preprocessing import LabelEncoder

# ——— MultiColumnLabelEncoder ———
class MultiColumnLabelEncoder:
    def __init__(self, columns=None):
        self.columns = columns

    def fit_transform(self, X):
        output = X.copy()
        cols = self.columns or X.columns
        for col in cols:
            output[col] = LabelEncoder().fit_transform(X[col])
        return output

# ——— Gower distance 정규화용 범위 계산 ———
def gowerRange(numeric_array):
    # 각 열별 (max - min)
    return np.ptp(numeric_array, axis=0)

# ——— 합성 데이터 → 실제 데이터 최소거리, 인덱스 리턴 ———
def get_ds(idx):
    # 범주형 부분
    tmp_cat = synth_data_cat[idx] != data_cat    # [N, n_cat]
    gdist_cat = tmp_cat.sum(axis=1)              # [N,]
    # 수치형 부분
    tmp_num = synth_data_num[idx] - data_num     # [N, n_num]
    tmp_num = np.abs(tmp_num) / ranges           # 정규화
    gdist_num = tmp_num.sum(axis=1)              # [N,]
    # 전체 컬럼 평균 거리
    dist = (gdist_cat + gdist_num) / n_columns   # [N,]
    min_idx = np.argmin(dist)
    return dist[min_idx], min_idx

# ——— 실제 데이터 간 최소거리 (자기 자신 제외) 리턴 ———
def get_do(idx):
    tmp_cat = data_cat[idx] != data_cat         # [N, n_cat]
    gdist_cat = tmp_cat.sum(axis=1)
    tmp_num = data_num[idx] - data_num          # [N, n_num]
    tmp_num = np.abs(tmp_num) / ranges
    gdist_num = tmp_num.sum(axis=1)
    dist = (gdist_cat + gdist_num) / n_columns  # [N,]

    # 자기 자신(dist[idx]) 제외하고 최소값
    if idx == 0:
        return np.min(dist[1:])
    elif idx == len(data_cat) - 1:
        return np.min(dist[:idx])
    else:
        min1 = np.min(dist[:idx])
        min2 = np.min(dist[idx+1:])
        return min(min1, min2)

# ——— 메인 프로그램 ———
if __name__ == "__main__":
    # 1) 실제 데이터 읽기 & 타입 지정
    dtypes = {
        'age':'int', 'workclass':'category', 'education':'category',
        'marital_status':'category','occupation':'category',
        'relationship':'category','race':'category','gender':'category',
        'capital_gain':'int','capital_loss':'int','hours_per_week':'int',
        'native_country':'category','income':'category'
    }
    columns = list(dtypes.keys())
    data = pd.read_csv('adultSalary.csv', names=columns).astype(dtypes)

    # 2) 합성 데이터(pickle) 읽기
    synth_path = 'df_synthpop.pickle'
    with open(synth_path, 'rb') as fr:
        synth_data = pickle.load(fr)

    # 3) 라벨 인코딩
    cat_cols = ['workclass','education','marital_status',
                'occupation','relationship','race','gender',
                'native_country','income']
    num_cols = ['age','capital_gain','capital_loss','hours_per_week']

    coder = MultiColumnLabelEncoder(columns=cat_cols)
    combined = pd.concat([data, synth_data], axis=0).reset_index(drop=True)
    encoded = coder.fit_transform(combined)

    data_enc   = encoded.iloc[:len(data), :]
    synth_enc  = encoded.iloc[len(data):, :]

    data_cat       = data_enc[cat_cols].to_numpy()
    data_num       = data_enc[num_cols].to_numpy()
    synth_data_cat = synth_enc[cat_cols].to_numpy()
    synth_data_num = synth_enc[num_cols].to_numpy()

    # 4) 거리 계산에 쓸 파라미터
    ranges    = gowerRange(data_num)
    n_columns = len(cat_cols) + len(num_cols)

    # 5) Originality 지표 계산
    cnt1 = 0  # ds < do
    cnt2 = 0  # ds > do

    start = time.time()
    for i in range(len(synth_data_cat)):
        ds, matched_idx = get_ds(i)
        do = get_do(matched_idx)
        if ds < do:
            cnt1 += 1
        elif ds > do:
            cnt2 += 1

    originality = (cnt1 / (cnt1 + cnt2) * 100) if (cnt1 + cnt2) else 0.0

    # 6) 결과 출력
    print(f"실행시간: {time.time() - start:.2f}s")
    print(f"Originality → cnt1={cnt1}, cnt2={cnt2}, score={originality:.2f}%")


실행시간: 51.35s
Originality → cnt1=26740, cnt2=12569, score=68.03%


In [29]:
# ——— 샘플 수 확인 ———
print(f"실제 데이터 샘플 수: {data_cat.shape[0]}, 범주 컬럼 수: {data_cat.shape[1]}")
print(f"합성 데이터 샘플 수: {synth_data_cat.shape[0]}, 범주 컬럼 수: {synth_data_cat.shape[1]}")
print(f"실제 수치형 컬럼 수: {data_num.shape[1]}")
print(f"합성 수치형 컬럼 수: {synth_data_num.shape[1]}")


실제 데이터 샘플 수: 48842, 범주 컬럼 수: 9
합성 데이터 샘플 수: 48842, 범주 컬럼 수: 9
실제 수치형 컬럼 수: 4
합성 수치형 컬럼 수: 4


In [48]:
import cupy as cp
import numpy as np
import time

# … (전처리: data_cat, data_num, synth_data_cat, synth_data_num, ranges, n_columns 준비) …

# 1) GPU 배열 (float32)
data_cat_gpu   = cp.asarray(data_cat)  # bool/int
data_num_gpu   = cp.asarray(data_num,    dtype=cp.float32)
synth_cat_gpu  = cp.asarray(synth_data_cat)
synth_num_gpu  = cp.asarray(synth_data_num, dtype=cp.float32)
ranges_gpu     = cp.asarray(ranges,       dtype=cp.float32)

Nd, Ns = data_cat_gpu.shape[0], synth_cat_gpu.shape[0] # 48842 48842
batch_data  = 1024   
batch_synth = 516
start_time = time.time()

# 2) data↔data 최소거리(do) 계산
do_gpu = cp.empty((Nd,), dtype=cp.float32)
for ds in range(0, Nd, batch_data):
    de = min(ds + batch_data, Nd)
    # [Bdata, Nd, C_cat]
    cat_d = (data_cat_gpu[ds:de,None,:] != data_cat_gpu[None,:,:])
    # [Bdata, Nd, C_num]
    num_d = cp.abs(data_num_gpu[ds:de,None,:] - data_num_gpu[None,:,:]) / ranges_gpu
    # 거리 합산 & 정규화
    dist  = (cat_d.sum(axis=2).astype(cp.float32)
           + num_d.sum(axis=2)) / n_columns  # [Bdata, Nd]
    # 자기 자신 제외
    idxs = cp.arange(ds, de)
    dist[cp.arange(de-ds), idxs] = cp.inf
    do_gpu[ds:de] = dist.min(axis=1)

# 3) synth↔data 최소거리(ds) + argmin(idx) 계산
ds_list, idx_list = [], []
for ss in range(0, Ns, batch_synth):
    se = min(ss + batch_synth, Ns)
    ds_batch  = cp.full((se-ss,), cp.inf, dtype=cp.float32)
    idx_batch = cp.zeros((se-ss,),        dtype=cp.int32)

    for ds in range(0, Nd, batch_data):
        de = min(ds + batch_data, Nd)
        cat_d = (synth_cat_gpu[ss:se,None,:] != data_cat_gpu[None,ds:de,:])
        num_d = cp.abs(synth_num_gpu[ss:se,None,:] - data_num_gpu[None,ds:de,:]) / ranges_gpu
        dist  = (cat_d.sum(axis=2).astype(cp.float32)
               + num_d.sum(axis=2)) / n_columns  # [Bsynth, Bdata]

        min_vals = dist.min(axis=1)
        min_idx  = dist.argmin(axis=1) + ds
        mask     = min_vals < ds_batch
        ds_batch[mask]  = min_vals[mask]
        idx_batch[mask] = min_idx[mask]

    ds_list.append(ds_batch)
    idx_list.append(idx_batch)

# 4) 한 번에 합치고 Originality 계산
ds_all  = cp.concatenate(ds_list)   # (Ns,)
idx_all = cp.concatenate(idx_list)  # (Ns,)

cnt1 = int(cp.sum(ds_all <  do_gpu[idx_all]))
cnt2 = int(cp.sum(ds_all >  do_gpu[idx_all]))
orig  = cnt1/(cnt1+cnt2)*100 if (cnt1+cnt2) else 0.0
gpu_time = time.time() - start_time
print(f"GPU 실행시간: {gpu_time:.2f}s")
print(f"GPU Originality → cnt1={cnt1}, cnt2={cnt2}, score={orig:.2f}%")


GPU 실행시간: 30.04s
GPU Originality → cnt1=26740, cnt2=12569, score=68.03%
